In [1]:
import matplotlib.pyplot as plt
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import numpy as np

In [2]:
import pandas as pd 
traindata = pd.read_csv("train.csv",  encoding='utf8' )
del traindata['Unnamed: 0']
traindata.head()

,id,text,label
0,547530,Beşiktaş'ın eski teknik direktörü Slaven Bilic...,spor
1,42683,"14 Şubat, Katolik Kilisesi’nin azizlerinden St...",yazarlar
2,770925,İstanbul Kağıthane’de gece yarısı sokakta oyun...,video
3,473224,Ziraat Türkiye Kupası maçında deplasmanda Amed...,video
4,44833,"Eski Brezilyalı yıldız oyuncu Ronaldo, Brezily...",spor


In [3]:
testdata = pd.read_csv("test.csv",  encoding='utf8' )
del testdata['Unnamed: 0']

X_train = traindata['text']
y_train = traindata['label']

X_test = testdata['text']
y_test = testdata['label']


In [4]:
from pickle import load
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.vis_utils import plot_model
from keras.models import Model
from keras.layers import Input
from keras.layers import Dropout
from keras.layers.merge import concatenate
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

# calculating the maximum document length
def max_length(lines):
	return max([len(s.split()) for s in lines])

In [5]:
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils

def encode_labels(y_train, y_test):
	le = LabelEncoder()
	le.fit(y_train)
	y_train_enc = le.transform(y_train)
	y_test_enc = le.transform(y_test)
	return y_train_enc, y_test_enc

y_train_enc, y_test_enc = encode_labels(y_train, y_test)
y_train_encoded_labels = np_utils.to_categorical(y_train_enc)

from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train_encoded_labels, test_size=0.2)

**Run up until this part then run whichever model you want**

Randomized Weights Static CNN

In [28]:

# define the model
def define_model(length, vocab_size):
  inputs1 = Input(shape=(length,))
  embedding1 = Embedding(vocab_size, 100, trainable = False)(inputs1)
  conv1 = Conv1D(filters=32, kernel_size=4, activation='relu')(embedding1)
  drop1 = Dropout(0.5)(conv1)
  pool1 = MaxPooling1D(pool_size=2)(drop1)
  flat1 = Flatten()(pool1)
  
  dense1 = Dense(10, activation='relu')(flat1)
  outputs = Dense(5, activation='softmax')(dense1)
  model = Model(inputs=[inputs1], outputs=outputs)
  
  # summarize
  print(model.summary())
  plot_model(model, show_shapes=True)
  return model



# calculating max document length
length = max_length(X_train)

tokenizer = Tokenizer()
X_train_tokens = tokenizer.fit_on_texts(X_train)
encoded = tokenizer.texts_to_sequences(X_train)
encoded2 = tokenizer.texts_to_sequences(X_val)

X_train = pad_sequences(encoded, maxlen=length, padding='post')
X_val = pad_sequences(encoded2, maxlen=length, padding='post')


# calculating vocabulary size
vocab_size = len(tokenizer.word_index) + 1
print('Max document length: %d' % length)
print('Vocabulary size: %d' % vocab_size)

# defining model
model = define_model(length, vocab_size)
# fitting model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train, y_train , validation_data=(X_val, y_val), epochs=5)


y_pred=np.argmax(model.predict(X_val), axis=-1) 
print("Classification Report:")
print(classification_report(y_val.argmax(axis=1),y_pred))

print("confusion matrix:")
print(confusion_matrix(y_val.argmax(axis=1),y_pred))

Max document length: 6994
Vocabulary size: 180206
Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 6994)]            0         
_________________________________________________________________
embedding_3 (Embedding)      (None, 6994, 100)         18020600  
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 6991, 32)          12832     
_________________________________________________________________
dropout_3 (Dropout)          (None, 6991, 32)          0         
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 3495, 32)          0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 111840)            0         
_________________________________________________________________
dense_6 (

Randomized Weights NonStatic CNN

In [6]:
# define the model
def define_model(length, vocab_size):
  inputs1 = Input(shape=(length,))
  embedding1 = Embedding(vocab_size, 100, trainable = True)(inputs1)
  conv1 = Conv1D(filters=32, kernel_size=4, activation='relu')(embedding1)
 
  drop1 = Dropout(0.5)(conv1)
  pool1 = MaxPooling1D(pool_size=2)(drop1)
  flat1 = Flatten()(pool1)
  
  dense1 = Dense(10, activation='relu')(flat1)
  outputs = Dense(5, activation='softmax')(dense1)
  model = Model(inputs=[inputs1], outputs=outputs)
  
  # summarize
  print(model.summary())
  plot_model(model, show_shapes=True)
  return model



# calculating max document length
length = max_length(X_train)

tokenizer = Tokenizer()
X_train_tokens = tokenizer.fit_on_texts(X_train)
encoded = tokenizer.texts_to_sequences(X_train)
encoded2 = tokenizer.texts_to_sequences(X_val)

X_train = pad_sequences(encoded, maxlen=length, padding='post')
X_val = pad_sequences(encoded2, maxlen=length, padding='post')


# calculating vocabulary size
vocab_size = len(tokenizer.word_index) + 1
print('Max document length: %d' % length)
print('Vocabulary size: %d' % vocab_size)

# defining model
model = define_model(length, vocab_size)
# fitting model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train, y_train , validation_data=(X_val, y_val), epochs=5)


y_pred=np.argmax(model.predict(X_val), axis=-1) 
print("Classification Report:")
print(classification_report(y_val.argmax(axis=1),y_pred))

print("confusion matrix:")
print(confusion_matrix(y_val.argmax(axis=1),y_pred))

Max document length: 4107
Vocabulary size: 181399
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 4107)]            0         
_________________________________________________________________
embedding (Embedding)        (None, 4107, 100)         18139900  
_________________________________________________________________
conv1d (Conv1D)              (None, 4104, 32)          12832     
_________________________________________________________________
dropout (Dropout)            (None, 4104, 32)          0         
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 2052, 32)          0         
_________________________________________________________________
flatten (Flatten)            (None, 65664)             0         
_________________________________________________________________
dense (Dens

**Run the following two parts for pretrained models then run whichever model you want**

In [6]:
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import gensim.models.keyedvectors as word2vec

from google.colab import drive
drive.mount('/content/drive')

pretrained_project2 = Word2Vec.load('/content/drive/MyDrive/word2vec/word2vecmodel.model')

trmodel = word2vec.KeyedVectors.load_word2vec_format('/content/drive/MyDrive/trmodel', binary= True)



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
from gensim import models
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()

X_train_tokens = tokenizer.fit_on_texts(X_train)
encoded = tokenizer.texts_to_sequences(X_train)
encoded2 = tokenizer.texts_to_sequences(X_val)

def get_weight_matrix(embedding, vocab, embedding_size):
  vocab_size = len(vocab) + 1
  weight_matrix = np.zeros((vocab_size, embedding_size))
  for word, i in vocab.items():
    try:
      weight_matrix[i] = embedding.wv.word_vec(word)
    except KeyError:
      weight_matrix[i]=np.random.normal(0,np.sqrt(0.25),embedding_size)
        
  return weight_matrix

embeddingmatrix =get_weight_matrix(pretrained_project2,tokenizer.word_index, 500) #embedding matrix for project2 word2vec

embeddingmatrix2 =get_weight_matrix(trmodel,tokenizer.word_index, 400) #embedding matrix for trmodel word2vec

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  


CNN with Pretrained Word Embeddings from Project02 

In [8]:
# defining the model
def define_model(length, vocab_size):

  inputs1 = Input(shape=(length,))
  embedding1 = Embedding(vocab_size, 500, weights= [embeddingmatrix])(inputs1)
  conv1 = Conv1D(filters=32, kernel_size=4, activation='relu')(embedding1)

  drop1 = Dropout(0.5)(conv1)
  pool1 = MaxPooling1D(pool_size=2)(drop1)
  flat1 = Flatten()(pool1)
  
  dense1 = Dense(10, activation='relu')(flat1)
  outputs = Dense(5, activation='softmax')(dense1)
  model = Model(inputs=[inputs1], outputs=outputs)
  
  print(model.summary())
  plot_model(model, show_shapes=True)
  return model



# calculating max document length
length = max_length(X_train)


X_train = pad_sequences(encoded, maxlen=length, padding='post')
X_val = pad_sequences(encoded2, maxlen=length, padding='post')


# calculating vocabulary size
vocab_size = len(tokenizer.word_index) + 1
print('Max document length: %d' % length)
print('Vocabulary size: %d' % vocab_size)

# define model
model = define_model(length, vocab_size)
# fit model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train, y_train , validation_data=(X_val, y_val), epochs=5)



y_pred=np.argmax(model.predict(X_val), axis=-1) 
print("Classification Report:")
print(classification_report(y_val.argmax(axis=1),y_pred))

print("confusion matrix:")
print(confusion_matrix(y_val.argmax(axis=1),y_pred))

Max document length: 6994
Vocabulary size: 181732
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 6994)]            0         
_________________________________________________________________
embedding (Embedding)        (None, 6994, 500)         90866000  
_________________________________________________________________
conv1d (Conv1D)              (None, 6991, 32)          64032     
_________________________________________________________________
dropout (Dropout)            (None, 6991, 32)          0         
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 3495, 32)          0         
_________________________________________________________________
flatten (Flatten)            (None, 111840)            0         
_________________________________________________________________
dense (Dens

CNN with Pretrained Word Embeddings from Turkish Word2Vec Model (Static)

In [8]:
# calculate the maximum document length
def max_length(lines):
	return max([len(s.split()) for s in lines])


# define the model
def define_model(length, vocab_size):
  # channel 1
  inputs1 = Input(shape=(length,))
  embedding1 = Embedding(vocab_size, 400, weights= [embeddingmatrix2], trainable = False)(inputs1)
  conv1 = Conv1D(filters=32, kernel_size=4, activation='relu')(embedding1)
  drop1 = Dropout(0.5)(conv1)
  pool1 = MaxPooling1D(pool_size=2)(drop1)
  flat1 = Flatten()(pool1)
  
  dense1 = Dense(10, activation='relu')(flat1)
  outputs = Dense(5, activation='softmax')(dense1)
  model = Model(inputs=[inputs1], outputs=outputs)
  
  # summarize
  print(model.summary())
  plot_model(model, show_shapes=True)
  return model



# calculate max document length
length = max_length(X_train)


X_train = pad_sequences(encoded, maxlen=length, padding='post')
X_val = pad_sequences(encoded2, maxlen=length, padding='post')


# calculate vocabulary size
vocab_size = len(tokenizer.word_index) + 1
print('Max document length: %d' % length)
print('Vocabulary size: %d' % vocab_size)

# define model
model = define_model(length, vocab_size)
# fit model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train, y_train , validation_data=(X_val, y_val), epochs=5)



y_pred=np.argmax(model.predict(X_val), axis=-1) 
print("Classification Report:")
print(classification_report(y_val.argmax(axis=1),y_pred))

print("confusion matrix:")
print(confusion_matrix(y_val.argmax(axis=1),y_pred))

Max document length: 4107
Vocabulary size: 180232
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 4107)]            0         
_________________________________________________________________
embedding (Embedding)        (None, 4107, 400)         72092800  
_________________________________________________________________
conv1d (Conv1D)              (None, 4104, 32)          51232     
_________________________________________________________________
dropout (Dropout)            (None, 4104, 32)          0         
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 2052, 32)          0         
_________________________________________________________________
flatten (Flatten)            (None, 65664)             0         
_________________________________________________________________
dense (Dens

CNN with Pretrained Word Embeddings from Turkish Word2Vec Model (Non Static)

In [16]:
# calculate the maximum document length
def max_length(lines):
	return max([len(s.split()) for s in lines])


# define the model
def define_model(length, vocab_size):
  # channel 1
  inputs1 = Input(shape=(length,))
  embedding1 = Embedding(vocab_size, 400, weights= [embeddingmatrix2], trainable = True)(inputs1)
  conv1 = Conv1D(filters=32, kernel_size=4, activation='relu')(embedding1)
  drop1 = Dropout(0.5)(conv1)
  pool1 = MaxPooling1D(pool_size=2)(drop1)
  flat1 = Flatten()(pool1)
  
  dense1 = Dense(10, activation='relu')(flat1)
  outputs = Dense(5, activation='softmax')(dense1)
  model = Model(inputs=[inputs1], outputs=outputs)
  
  # summarize
  print(model.summary())
  plot_model(model, show_shapes=True)
  return model



# calculate max document length
length = max_length(X_train)


X_train = pad_sequences(encoded, maxlen=length, padding='post')
X_val = pad_sequences(encoded2, maxlen=length, padding='post')


# calculate vocabulary size
vocab_size = len(tokenizer.word_index) + 1
print('Max document length: %d' % length)
print('Vocabulary size: %d' % vocab_size)

# define model
model = define_model(length, vocab_size)
# fit model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train, y_train , validation_data=(X_val, y_val), epochs=5)


y_pred=np.argmax(model.predict(X_val), axis=-1) 
print("Classification Report:")
print(classification_report(y_val.argmax(axis=1),y_pred))

print("confusion matrix:")
print(confusion_matrix(y_val.argmax(axis=1),y_pred))

Max document length: 6994
Vocabulary size: 180888
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 6994)]            0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 6994, 400)         72355200  
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 6991, 32)          51232     
_________________________________________________________________
dropout_1 (Dropout)          (None, 6991, 32)          0         
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 3495, 32)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 111840)            0         
_________________________________________________________________
dense_2 (

For each model trial the code must be run from the beggining